<a href="https://colab.research.google.com/github/Manuel-Gomez-05/SenalesySistemas2/blob/main/Parcial_2_Manuel_Gomez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 2: Señales y Sistemas 2025-II
**Profesor:** Andrés Marino Álvarez Meza, Ph.D.

**Estudiante:** Manuel Alejandro Gómez
**Fecha:** Diciembre 2025

---

## 1. Justificación Teórica y Procedimiento

### Punto 1: Modulación de Amplitud (DSB-SC)

En este punto se simula un sistema de comunicación de Doble Banda Lateral con Portadora Suprimida (DSB-SC). El objetivo es visualizar y escuchar cómo la información se transforma en cada etapa.

**A. Modelo Matemático:**

1. **Modulación:** La señal de audio (mensaje) $m(t)$ se traslada a alta frecuencia multiplicándola por una portadora sinusoidal:

$$r(t) = m(t) \cdot \cos(2\pi f_c t)$$

Esto genera dos bandas laterales centradas en $\pm f_c$, eliminando la componente DC.

2. **Demodulación Coherente (Mezcla):** Para recuperar el mensaje, multiplicamos la señal recibida nuevamente por la misma portadora (asumiendo sincronización perfecta $\theta=0$):

$$x(t) = r(t) \cdot \cos(2\pi f_c t) = m(t) \cos^2(2\pi f_c t)$$

Aplicando identidades trigonométricas, obtenemos:

$$x(t) = \frac{1}{2}m(t) + \frac{1}{2}m(t)\cos(4\pi f_c t)$$

3.  **Filtrado (FFT):** La señal resultante tiene el audio original (banda base) y una réplica en $2f_c$ (alta frecuencia). Para aislar el mensaje:
    * Se lleva la señal al dominio de la frecuencia usando la **FFT**.
    * Se aplica una **máscara ideal** (filtro rectangular) que vuelve cero todas las frecuencias mayores a la frecuencia de corte ($f_{corte}$).
    * Se aplica la **IFFT** para recuperar la señal filtrada en el tiempo.

---

### Punto 2: Sistemas Dinámicos y Equivalencia RLC

El objetivo es relacionar un sistema mecánico (Masa-Resorte-Amortiguador) con un circuito eléctrico RLC para que tengan la misma respuesta transitoria (mismo $\zeta$ y $\omega_n$).

**A. Sistema Mecánico:**
La dinámica está regida por:
$$H_{mec}(s) = \frac{1}{ms^2 + cs + k}$$
Donde la frecuencia natural es $\omega_n = \sqrt{k/m}$ y el amortiguamiento $\zeta = \frac{c}{2\sqrt{km}}$.

**B. Equivalencia Eléctrica:**
Se modela un circuito con un Inductor ($L$) en serie con un paralelo Resistencia-Capacitor ($RC$). Igualando los coeficientes de la ecuación característica del circuito con la mecánica:

1.  Término independiente: $\frac{k}{m} = \frac{1}{LC} \implies L = \frac{m}{k \cdot C}$
2.  Término lineal: $\frac{c}{m} = \frac{1}{RC} \implies R = \frac{m}{c \cdot C}$

**C. Implementación en el Dashboard:**
Dado que hay 3 incógnitas eléctricas ($R, L, C$) y solo 2 ecuaciones mecánicas, el sistema es indeterminado.
* **Solución:** Se fija el valor del **Capacitor ($C$)** como un parámetro de referencia ajustable por el usuario (o un valor comercial estándar).
* **Cálculo Dinámico:** Al variar la masa $m$, el resorte $k$ o el amortiguador $c$, el código recalcula instantáneamente $R$ y $L$ para mantener la equivalencia exacta.
* **Lazo Cerrado:** Si se activa el control proporcional ($K_p$), la "rigidez" del sistema aumenta a $k_{eq} = k + K_p$, lo cual se refleja inmediatamente en el cálculo de los componentes eléctricos equivalentes.

In [46]:
print("Instalando librerías necesarias... (Esto puede tardar unos segundos)")
!pip install streamlit numpy scipy matplotlib pandas soundfile pydub -q
# Instalamos la versión de desarrollo de yt-dlp para evitar errores de YouTube
!pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz -q
!apt-get install ffmpeg -y -qq
print("✅ Instalación completada.")

Instalando librerías necesarias... (Esto puede tardar unos segundos)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ Instalación completada.


In [47]:
import os
if not os.path.exists("pages"):
    os.makedirs("pages")
print("✅ Estructura de carpetas creada.")

✅ Estructura de carpetas creada.


In [48]:
%%writefile presentacion.py
import streamlit as st

st.set_page_config(
    page_title="Parcial 2 - SyS",
    page_icon="📡",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.title("📡 Parcial 2: Señales y Sistemas 2025-II")
st.markdown("**Profesor:** Andrés Marino Álvarez Meza, Ph.D.")
st.markdown("### **Estudiante:** Manuel Alejandro Gómez")
st.divider()

# Layout de dos columnas para la descripción detallada
col1, col2 = st.columns(2)

with col1:
    st.info("### 📻 Punto 1: Modulación AM (DSB-SC)")
    st.markdown("""
    Este componente interactivo del dashboard, construido con Streamlit, facilita la visualización y comprensión del proceso de **modulación y demodulación de Doble Banda Lateral con Portadora Suprimida (DSB-SC)**.

    Se permite al usuario seleccionar la señal mensaje a utilizar, ofreciendo dos opciones:
    * i) un **tono de prueba sintético** (senoidal amortiguada), o
    * ii) un **segmento de audio real**, cargado desde un enlace de YouTube de forma robusta.

    La implementación de los filtros necesarios para la etapa de demodulación se realiza utilizando **técnicas espectrales mediante la FFT** (máscara ideal en frecuencia), lo que garantiza una eliminación precisa de las componentes de alta frecuencia.

    El dashboard está diseñado para describir de manera **concreta y clara** cada una de las fases involucradas. Presenta **gráficas relevantes** de las señales en sus etapas intermedias (Modulación, Mezcla, Filtrado), mostrando su representación tanto en el **dominio del tiempo** como en el **dominio de la frecuencia (espectro)**, acompañado de la reproducción de audio en cada etapa.
    """)

with col2:
    st.warning("### 🏎️ Punto 2: Sistemas Dinámicos y Control")
    st.markdown(r"""
    Mediante la plataforma Streamlit, se ha desarrollado un entorno interactivo para la simulación y análisis de sistemas dinámicos y su equivalencia eléctrica.

    El usuario tiene la capacidad de definir las características clave del sistema de segundo orden: el **tipo de respuesta** (subamortiguada, crítica o sobreamortiguada) ajustando dinámicamente los parámetros físicos ($m, k, c$).

    Este módulo proporciona visualizaciones completas para la configuración en **lazo abierto** y **lazo cerrado** (con control $K_p$), incluyendo:

    * El **diagrama de Bode** (magnitud y fase) para analizar la respuesta en frecuencia.
    * El **mapa de polos y ceros** en el plano complejo $s$ para comprender la estabilidad.
    * Las **respuestas temporales** a entradas estándar: impulso, escalón unitario y rampa.
    * El cálculo y presentación en tiempo real de **parámetros temporales** clave: tiempo de levantamiento ($t_r$), tiempo de establecimiento ($t_s$) y frecuencia natural ($\omega_n$).
    * **Equivalencia RLC:** Estimación automática de los valores de componentes eléctricos ($R, L$) necesarios para replicar la dinámica mecánica, permitiendo al usuario fijar un Capacitor ($C$) de referencia.
    """)

st.markdown("---")
st.caption("Dashboard Interactivo de Ingeniería | Modo Oscuro Activado 🌑")

Overwriting presentacion.py


In [49]:
%%writefile pages/punto1_modulacion.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq, fftshift
import os

# Configuración
plt.style.use('dark_background')
st.set_page_config(page_title="Punto 1", layout="wide")

# Intento de importación segura
try:
    import yt_dlp
    from pydub import AudioSegment
except ImportError:
    st.error("Librerías faltantes. Por favor reinstala las dependencias.")

@st.cache_data(show_spinner=False)
def download_audio_safe(url, fs=44100):
    filename = "temp.wav"
    try:
        # Configuración para evitar bloqueo 403 de YouTube
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': '%(id)s.%(ext)s',
            'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'wav'}],
            'quiet': True, 'no_warnings': True, 'nocheckcertificate': True,
            'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            temp_name = f"{info['id']}.wav"

        audio = AudioSegment.from_file(temp_name)
        audio = audio.set_channels(1).set_frame_rate(fs)
        # Recorte a 5 segundos (segundo 15 al 20)
        start = 15000 if len(audio) > 20000 else 0
        audio = audio[start:start+5000]

        data = np.array(audio.get_array_of_samples()).astype(np.float32)
        if np.max(np.abs(data)) > 0: data /= np.max(np.abs(data))

        if os.path.exists(temp_name): os.remove(temp_name)
        return fs, data, None
    except Exception as e:
        return None, None, str(e)

def plot_spec(t, y, fs, title, col):
    if y is None: return
    N = len(y)
    Y = fftshift(fft(y))
    f = fftshift(fftfreq(N, 1/fs))
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
    ax1.plot(t, y, color=col, lw=0.8); ax1.set_title(f"{title} (Tiempo)"); ax1.grid(alpha=0.2)
    ax2.plot(f, np.abs(Y)/N, color='#FF00FF', lw=0.8); ax2.set_title("Espectro"); ax2.set_xlim([-fs/2, fs/2]); ax2.grid(alpha=0.2)
    st.pyplot(fig)

st.title("🎹 1. Modulación AM (DSB-SC)")
st.markdown("**Estudiante:** Manuel Alejandro Gómez")

with st.sidebar:
    st.header("🎛️ Configuración")
    src = st.radio("Fuente", ["Tono de Prueba", "YouTube"])
    fc = st.slider("Portadora fc", 5000, 15000, 10000, 1000)
    cut = st.slider("Corte LPF", 1000, 6000, 4000, 500)

fs = 44100
audio = None

if src == "YouTube":
    url = st.text_input("URL YouTube", "https://www.youtube.com/watch?v=kJQP7kiw5Fk")
    if st.button("Descargar"):
        with st.spinner("Descargando..."):
            _, audio, err = download_audio_safe(url)
            if err: st.error(f"Error: {err}")
else:
    t = np.linspace(0, 3, 3*fs)
    audio = 0.5*np.cos(2*np.pi*440*t)*np.exp(-t) + 0.3*np.cos(2*np.pi*800*t)

if audio is not None:
    t = np.linspace(0, len(audio)/fs, len(audio))
    st.markdown("### 1. Original"); st.audio(audio, sample_rate=fs); plot_spec(t, audio, fs, "Mensaje", '#00FF00')

    st.markdown("### 2. Modulada"); carrier = np.cos(2*np.pi*fc*t); mod = audio*carrier
    st.audio(mod, sample_rate=fs); plot_spec(t, mod, fs, "Modulada", '#FFFF00')

    st.markdown("### 3. Mezcla"); mix = mod*carrier
    st.audio(mix, sample_rate=fs); plot_spec(t, mix, fs, "Mixer", '#00FFFF')

    st.markdown("### 4. Recuperada"); Y = fft(mix); f = fftfreq(len(mix), 1/fs); mask = np.abs(f) < cut
    rec = ifft(Y*mask).real * 2
    st.audio(rec, sample_rate=fs); plot_spec(t, rec, fs, "Final", '#FF00FF')

Overwriting pages/punto1_modulacion.py


In [50]:
%%writefile pages/punto2_sistemas.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

plt.style.use('dark_background')
st.set_page_config(page_title="Punto 2", layout="wide")

st.title("🏎️ 2. Sistemas Dinámicos y RLC")
st.markdown("**Estudiante:** Manuel Alejandro Gómez")

col1, col2 = st.columns([1, 2.5])

with col1:
    st.markdown("### ⚙️ Mecánico (Inputs)")
    m = st.number_input("Masa (kg)", 0.1, 50.0, 1.0, 0.1)
    k = st.number_input("Resorte k (N/m)", 1.0, 500.0, 25.0, 1.0)

    cc = 2*np.sqrt(k*m)
    st.caption(f"Amort. Crítico: {cc:.2f}")
    tipo = st.selectbox("Preset", ["Subamortiguado", "Crítico", "Sobreamortiguado"])
    val_c = cc*0.3 if "Sub" in tipo else (cc if "Crítico" in tipo else cc*2.0)
    c = st.slider("Amortiguador c", 0.0, cc*3, val_c, 0.1)

    st.divider()
    st.markdown("### 🎛️ Control")
    lazo = st.radio("Lazo", ["Abierto", "Cerrado (P)"])
    Kp = 0.0
    if "Cerrado" in lazo: Kp = st.slider("Ganancia Kp", 1.0, 200.0, 50.0)

    st.divider()
    st.markdown("### 🔋 Eléctrico (Referencia)")
    C_uf = st.slider("Capacitor C (µF)", 10.0, 1000.0, 100.0, 10.0)

# CÁLCULOS REACTIVOS
k_tot = k + Kp if "Cerrado" in lazo else k
den = [m, c, k_tot]; num = [Kp] if "Cerrado" in lazo else [1]
sys = signal.TransferFunction(num, den)

wn = np.sqrt(k_tot/m); zeta = c/(2*np.sqrt(k_tot*m))

# RLC EQUIVALENTE (Se actualiza al mover cualquier slider)
C_elec = C_uf * 1e-6
L_elec = m / (k_tot * C_elec)
R_elec = m / (c * C_elec) if c > 1e-5 else np.inf

# SIMULACION
t_fin = 20 if wn < 1 else 50/wn
t = np.linspace(0, t_fin, 2000)
ts_vec, ys = signal.step(sys, T=t)
fv = ys[-1]

# Métricas
bounds = np.where((ys < fv*0.98) | (ys > fv*1.02))[0]
ts_val = f"{ts_vec[bounds[-1]]:.3f} s" if len(bounds)>0 else "∞"

if zeta < 1:
    crs = np.where(ys >= fv)[0]
    tr_val = f"{ts_vec[crs[0]]:.3f} s" if len(crs)>0 else "> Sim"
else:
    i10 = np.argmax(ys>=0.1*fv); i90 = np.argmax(ys>=0.9*fv)
    tr_val = f"{ts_vec[i90]-ts_vec[i10]:.3f} s"

with col2:
    st.markdown("#### ⚡ Circuito Equivalente (Calculado)")
    st.info(f"Para igualar la dinámica (con k_eq={k_tot:.1f}), usa estos componentes:")
    c_a, c_b, c_c = st.columns(3)
    c_a.metric("Resistencia R", f"{R_elec:.2f} Ω")
    c_b.metric("Inductancia L", f"{L_elec:.4f} H")
    c_c.metric("Capacitor C", f"{C_uf:.0f} µF")

    st.divider()
    st.markdown("#### 📊 Respuesta Dinámica")
    d1, d2, d3, d4 = st.columns(4)
    d1.metric("Zeta", f"{zeta:.3f}"); d2.metric("Wn", f"{wn:.2f}")
    d3.metric("Ts (2%)", ts_val); d4.metric("Tr (Subida)", tr_val)

    # ---------------------------------------------------------
    # DIAGRAMA DE BODE
    # ---------------------------------------------------------
    tab1, tab2, tab3 = st.tabs(["📈 Tiempo", "📉 Diagrama Bode", "❌ Polos"])

    with tab1:
        u = st.radio("Entrada", ["Escalón", "Impulso", "Rampa"], horizontal=True)
        fig, ax = plt.subplots(figsize=(10, 3.5))
        if "Escalón" in u: ax.plot(ts_vec, ys, 'cyan'); ax.axhline(fv, color='w', ls='--', alpha=0.5)
        elif "Impulso" in u: ti, yi = signal.impulse(sys, T=t); ax.plot(ti, yi, 'magenta')
        else: tr, yr, _ = signal.lsim(sys, U=t, T=t); ax.plot(tr, yr, 'yellow'); ax.plot(tr, tr, 'w', ls=':', alpha=0.3)
        ax.grid(alpha=0.15); st.pyplot(fig)

    with tab2: # Lógica del Diagrama de Bode
        w, mag, phase = signal.bode(sys)
        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 4))
        # Magnitud
        ax1.semilogx(w, mag, '#00FF00')
        ax1.set_ylabel("Magnitud (dB)")
        ax1.grid(True, alpha=0.2)
        ax1.set_title("Respuesta en Frecuencia (Bode)")
        # Fase
        ax2.semilogx(w, phase, '#00FFFF')
        ax2.set_ylabel("Fase (deg)")
        ax2.set_xlabel("Frecuencia (rad/s)")
        ax2.grid(True, alpha=0.2)
        st.pyplot(fig)

    with tab3:
        fig, ax = plt.subplots(figsize=(4,4))
        ax.scatter(np.real(sys.poles), np.imag(sys.poles), c='r', marker='x', s=100)
        ax.axhline(0, color='w', alpha=0.3); ax.axvline(0, color='w', alpha=0.3); ax.grid(alpha=0.2)
        st.pyplot(fig)

Overwriting pages/punto2_sistemas.py


In [51]:
import subprocess
import time
import re
import os
import sys

print("--- 🚀 Iniciando Servidor... ---")

# 1. Instalar Cloudflared
if not os.path.exists("cloudflared"):
    !curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
    !chmod +x cloudflared

# 2. Correr Streamlit
with open("streamlit.log", "w") as f:
    proc_st = subprocess.Popen(["streamlit", "run", "presentacion.py", "--server.port", "8501", "--server.address", "0.0.0.0", "--theme.base", "dark"], stdout=f, stderr=f)

# 3. Correr Tunel (Borrando log anterior)
if os.path.exists("tunnel.log"): os.remove("tunnel.log")
with open("tunnel.log", "w") as f:
    proc_cf = subprocess.Popen(["./cloudflared", "tunnel", "--url", "http://localhost:8501"], stdout=f, stderr=subprocess.STDOUT)

print("⏳ Esperando URL pública (aprox 10s)...")
url = None
for i in range(30):
    time.sleep(1)
    try:
        if os.path.exists("tunnel.log"):
            with open("tunnel.log", "r") as f:
                content = f.read()
                match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
                if match:
                    url = match.group(0)
                    break
    except: pass

if url:
    print(f"\n✅  LINK DEL DASHBOARD:  {url}  \n")
else:
    print("⚠️ No se encontró link. Revisa tunnel.log")
    !tail -n 10 tunnel.log

# Loop para mantener vivo
try:
    while True: time.sleep(1)
except KeyboardInterrupt:
    proc_st.terminate()
    proc_cf.terminate()

--- 🚀 Iniciando Servidor... ---
⏳ Esperando URL pública (aprox 10s)...

✅  LINK DEL DASHBOARD:  https://beijing-listprice-thick-magic.trycloudflare.com  

